# SET UP

## CREAR EL ENTORNO PARA EL PROYECTO

Crear un nuevo entorno e instalar los paquetes básicos:

1. Ir al prompt o terminal
2. Copiar y poner la línea inferior:

conda create --name leadscoring python numpy pandas matplotlib seaborn scikit-learn scipy sqlalchemy xgboost jupyter

3. Activar el nuevo entorno:

conda activate leadscoring

4. Instalar los paquetes que están en otros canales:

conda install -c conda-forge pyjanitor scikit-plot yellowbrick imbalanced-learn jupyter_contrib_nbextensions cloudpickle

conda install -c districtdatalabs yellowbrick

pip install category_encoders

5. Crear el environment.yml

conda env export > leadscoring.yml

#### Notas explicativas

- Creo un entorno ya que se necesita en produccion que sea un entorno con las mismas versiones con las cuales se entreno el modelo, sino cuando cambie la version de algun paquete puedo tener problemas en produccion.

## IMPORTAR PAQUETES

In [1]:
import os
import numpy as np
import pandas as pd

#Automcompletar rápido
%config IPCompleter.greedy=True

## CREAR EL DIRECTORIO DEL PROYECTO

### Definir el directorio raiz donde vamos a crear el proyecto

In [2]:
raiz = 'volumes/STEVEN/PROYECTOS'

### Definir el nombre del proyecto

In [3]:
nombre_dir = '01_Lead_Scoring'

### Crear el directorio y la estructura del proyecto

In [4]:
# Ruta base donde se crearán las carpetas
path = os.path.expanduser('~/'+raiz+nombre_dir)

# Lista de nombres de las carpetas a crear
carpetas = [
    '01_Documentos',
    '02_Datos',
    '02_Datos/01_Originales',
    '02_Datos/02_Validacion',
    '02_Datos/03_Trabajo',
    '02_Datos/04_Caches',
    '03_Notebooks',
    '03_Notebooks/01_Funciones',
    '03_Notebooks/02_Desarrollo',
    '03_Notebooks/03_Sistema',
    '04_Modelos',
    '05_Resultados',
    '09_Otros'
]

# Intentar crear las carpetas
try:
    for carpeta in carpetas:
        ruta_carpeta = os.path.join(path, carpeta)
        os.makedirs(ruta_carpeta, exist_ok=True)

    # Confirmar que ha funcionado bien
    print("Se ha creado satisfactoriamente el directorio %s " % path)
except OSError:
    print("La creación del directorio %s ha fallado" % path)

Se ha creado satisfactoriamente el directorio /Users/stevenallus/Documents/MAC_Google_Drive/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/01_Lead_Scoring 


### Situar el directorio de trabajo en el proyecto

In [5]:
os.chdir(path)

### Mover el environment.yml

Buscar manualmente el archivo leadscoring.yml, y moverlo a la carpeta /01_Documentos'

## CREAR LOS DATASETS INICIALES

Poner los datos originales en la carpeta '/02_Datos/01_Originales'

### Importar los datos

Sustituir el nombre_fichero_datos.

In [6]:
nombre_fichero_datos = 'leads.csv'

Cargar los datos.

In [7]:
ruta_completa = path + '/02_Datos/01_Originales/' + nombre_fichero_datos

datos = pd.read_csv(ruta_completa,index_col='id', sep = ';')
datos

,origen,fuente,no_enviar_email,no_llamar,compra,visitas_total,tiempo_en_site_total,paginas_vistas_visita,ult_actividad,ambito,ocupacion,conociste_google,conociste_revista,conociste_periodico,conociste_youtube,conociste_facebook,conociste_referencias,score_actividad,score_perfil,descarga_lm
id,,,,,,,,,,,,,,,,,,,,
660737,API,Chat,No,No,0,0.0,0,0.00,Page Visited on Website,Select,Unemployed,No,No,No,No,No,No,15.0,15.0,No
660728,API,Organic Search,No,No,0,5.0,674,2.50,Email Opened,Select,Unemployed,No,No,No,No,No,No,15.0,15.0,No
660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.00,Email Opened,Business Administration,Student,No,No,No,No,No,No,14.0,20.0,Yes
660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.00,Unreachable,Media and Advertising,Unemployed,No,No,No,No,No,No,13.0,17.0,No
660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.00,Converted to Lead,Select,Unemployed,No,No,No,No,No,No,15.0,18.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579564,Landing Page Submission,Direct Traffic,Yes,No,1,8.0,1845,2.67,Email Marked Spam,IT Projects Management,Unemployed,No,No,No,No,No,No,15.0,17.0,No
579546,Landing Page Submission,Direct Traffic,No,No,0,2.0,238,2.00,SMS Sent,Media and Advertising,Unemployed,No,No,No,No,No,No,14.0,19.0,Yes
579545,Landing Page Submission,Direct Traffic,Yes,No,0,2.0,199,2.00,SMS Sent,Business Administration,Unemployed,No,No,No,No,No,No,13.0,20.0,Yes


### Extraer y reservar el dataset de validación

In [8]:
val= datos.sample(frac = 0.3)

In [9]:
nombre_fichero_validacion = 'validacion.csv'

ruta_completa = path + '/02_Datos/02_Validacion/' + nombre_fichero_validacion

val.to_csv(ruta_completa)

### Extraer y guardar el dataset de trabajo

In [10]:
trabajo = datos.loc[~datos.index.isin(val.index)]

In [11]:
nombre_fichero_trabajo = 'trabajo.csv'

ruta_completa = path + '/02_Datos/03_Trabajo/' + nombre_fichero_trabajo

trabajo.to_csv(ruta_completa)

#### Nota

No cojo una muestra del dataset

Tengo pocos datos, no tiene sentido hacer esto, ahora si tengo una gran cantidad de datos entonces si que me interesa extraer una muestra donde voy a hacer validacion cruzada, train test, etc... Si es computacionalmente intenso lo que puedo hacer es coger esa muestra.